In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer # for stemming
from nltk.stem import WordNetLemmatizer # for lemmatization
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split , cross_val_score,KFold
from collections import Counter


from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
import warnings
!pip install imblearn
import imblearn
warnings.filterwarnings("ignore")

In [ ]:
!pip install keras
!pip install tensorflow


In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers import Embedding
from keras.models import Model
import string
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from keras.models import Sequential
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Bidirectional

#%matplotlib inline


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:

#df = pd.read_fwf('/content/drive/MyDrive/temp_files/hate and offensive/train.txt', sep=" ", header=None, names=[ "id", "label", "tweet"] , encoding='UTF-8')
df = pd.read_fwf('train.txt', sep=" ", header=None, names=[ "id", "label", "tweet"] , encoding='UTF-8')
# data=data[900:1100]
data_text = df[['tweet']]
data_text = data_text.astype('str')

tweets = []
for index, row in data_text.iterrows():
    tweets.append(row['tweet'])

print('Size of dataset:',len(tweets))

In [ ]:
df['label'].unique

In [ ]:
x = df['tweet']
y = df['label']

In [ ]:
token = Tokenizer()

In [ ]:
token.fit_on_texts(x)
seq = token.texts_to_sequences(x)

In [ ]:
pad_seq = pad_sequences(seq,maxlen=300)
vocab_size = len(token.word_index)+1
x = df['tweet']
y = df['label']

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
from keras.initializers import Constant
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer

#from keras.preprocessing.sequence import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
embedding_vector = {}
f=open("glove.6B.100d.txt",encoding="utf-8")



In [ ]:
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

In [ ]:
embedding_matrix = np.zeros((vocab_size,100))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

In [ ]:
y = df['label']
counter = Counter(y)
for k,v in counter.items():
 per = v / len(y) * 100
 print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

In [ ]:
modelLSTMGL = Sequential()
modelLSTMGL.add(Embedding(vocab_size,100,weights = [embedding_matrix],input_length=300,trainable = False))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad_seq, y, test_size=0.15,random_state=0)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers import Embedding,Bidirectional
from keras.models import Model


In [ ]:
modelLSTMGL = Sequential()
modelLSTMGL.add(Embedding(vocab_size,100,weights = [embedding_matrix],input_length=300,trainable = False))
modelLSTMGL.add(Bidirectional(LSTM(25)))
modelLSTMGL.add(Dense(16,activation = 'relu'))
modelLSTMGL.add(Dense(1,activation = 'sigmoid'))
modelLSTMGL.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])
history = modelLSTMGL.fit(pad_seq, y,epochs = 5,batch_size=32,validation_split=0.15)

In [ ]:
values = history.history

In [ ]:

val_loss = values['val_loss']
training_loss = values['loss']
training_acc = values['accuracy']
validation_acc = values['val_accuracy']
epochs = range(5)

plt.plot(epochs,val_loss,label = 'Validation Loss')
plt.plot(epochs,training_loss,label = 'Training Loss')
plt.title('LSTM+GLOVE Loss')
plt.legend()
plt.show()
plt.plot(epochs,validation_acc,label = 'Validation Accuracy')
plt.plot(epochs,training_acc,label = 'Training Accuracy')
plt.title('LSTM+GLOVE Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_score,recall_score,accuracy_score, f1_score
def show_confusion(label,pred_label,nm):
  class_names=['No-Addiction','Addiction']
  cm=confusion_matrix(label,pred_label)
  disp =ConfusionMatrixDisplay(cm,display_labels=class_names)
  disp.plot()
  plt.title(f'Confusion Matrix {nm} ')

  plt.xlabel('Actual labels')
  plt.ylabel('Predicted labels')
  plt.show()

def mymetrics(Y,P,alg = None):
    recall= recall_score(Y, P)
    acc2= accuracy_score(Y, P)
    precision= precision_score(Y, P)
    Fmeasure = f1_score(Y, P)
    print(f"accuracy = {acc2*100}")
    print(f'recall= {recall*100}')
    print(f'precision= {precision*100}')
    print(f'Fmeasure= {Fmeasure*100}')

In [ ]:
ptrLstm = modelLSTMGL.predict(X_train)
ptsLstm = modelLSTMGL.predict(X_test)
ptrLstm=(ptrLstm>0.5)
ptsLstm=(ptsLstm>0.5)
ptrLstm=ptrLstm.astype(float)
ptsLstm=ptsLstm.astype(float)



In [ ]:
from sklearn.metrics import classification_report

In [ ]:
c = classification_report(y_test,ptsLstm,digits=4)

print(c)

In [ ]:
print("===== : Validation Lstm+Glove : =====")
mymetrics(y_test,ptsLstm,'Validation Lstm+Glove')
show_confusion(ptsLstm,y_test,'Validation Lstm+Glove')

In [ ]:

#testing = pd.read_fwf('/content/drive/MyDrive/temp_files/hate and offensive/test.txt', sep=" ", header=None, names=[ "id", "label", "tweet"] , encoding='UTF-8')
testing = pd.read_fwf('test.txt', sep=" ", header=None, names=[ "id","label", "tweet"] , encoding='utf-8')

testing.head(10)


In [ ]:
x_test = testing['tweet']


In [ ]:
x_test = token.texts_to_sequences(x_test)

In [ ]:
testing_seq = pad_sequences(x_test,maxlen=300)


In [ ]:
predic = modelLSTMGL.predict(testing_seq)

In [ ]:
# preds=[]
# for i in predict:
#   preds.append(0 if i<0.5 else 1)
preds=(predic>0.5)
preds=preds.astype(float)

In [ ]:
z = classification_report(testing['label'], preds,digits=4)

print(z)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print()
print(" precision: {}".format(precision_score(testing['label'], preds)))
print(" recall: {}".format(recall_score(testing['label'], preds)))
print(" Accuracy: {}".format(accuracy_score(testing['label'], preds)))
print(" f1: {}".format(f1_score(testing['label'], preds)))

In [ ]:
print("===== : Test Lstm+Glove : =====")
mymetrics(preds,testing['label'])
show_confusion(preds,testing['label'] ,'Test Lstm+Glove')

In [ ]:
submit_df = pd.DataFrame({"id": testing["id"], "prediction": testing['label']})
submit_df.to_csv("submission.csv", index=False)